In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_train = pd.read_csv("./dataset/training_set_VU_DM.csv")
df_test = pd.read_csv("./dataset/test_set_VU_DM.csv")

# Get only the bets inv column (if any other company has better deals
inv_columns = [f'comp{i}_rate' for i in range(1, 9)]
# Get only the best rate (the lowest)
rate_columns = [f'comp{i}_rate_percent_diff' for i in range(1, 9)]

# Compute the maximum value across these columns for each row
df_train['comp_inv'] = df_train[inv_columns].max(axis=1)
df_test['comp_inv'] = df_test[inv_columns].max(axis=1)
df_train['comp_rate_percent_diff'] = df_train[rate_columns].max(axis=1)
df_test['comp_rate_percent_diff'] = df_train[rate_columns].max(axis=1)

drop_competitors_cols = [ 'date_time', 'visitor_hist_starrating', 'visitor_hist_adr_usd',
    'srch_booking_window',
    'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 
    'comp2_rate', 'comp2_inv','comp2_rate_percent_diff', 
    'comp3_rate', 'comp3_inv','comp3_rate_percent_diff', 
    'comp4_rate', 'comp4_inv','comp4_rate_percent_diff', 
    'comp5_rate', 'comp5_inv','comp5_rate_percent_diff', 
    'comp6_rate', 'comp6_inv','comp6_rate_percent_diff', 
    'comp7_rate', 'comp7_inv','comp7_rate_percent_diff', 
    'comp8_rate', 'comp8_inv','comp8_rate_percent_diff'
    ]

df_train = df_train.drop(columns=drop_competitors_cols)
df_train = df_train.drop(column="gross_bookings_usd")
df_test = df_test.drop(columns=drop_competitors_cols)

# Min Max scale
scaler = MinMaxScaler(feature_range=(0, 5))
df_train['prop_starrating'] = scaler.fit_transform(df_train['prop_starrating'])
df_test['prop_starrating'] = scaler.fit_transform(df_test['prop_starrating'])
df_train['prop_review_score'] = scaler.fit_transform(df_train['prop_review_score'])
df_test['prop_review_score'] = scaler.fit_transform(df_test['prop_review_score'])

max_feat = max(df_test["prop_location_score1"].max(), df_train["prop_location_score1"].max())
min_feat = min(df_test["prop_location_score1"].min(), df_train["prop_location_score1"].min())
scaler = MinMaxScaler(feature_range=(min_feat, max_feat))
df_train['prop_location_score1'] = scaler.fit_transform(df_train['prop_location_score1'])
df_test['prop_location_score1'] = scaler.fit_transform(df_test['prop_location_score1'])

max_feat = max(df_test["prop_log_historical_price"].max(), df_train["prop_log_historical_price"].max())
min_feat = max(df_test["prop_log_historical_price"].min(), df_train["prop_log_historical_price"].min())
scaler = MinMaxScaler(feature_range=(min_feat, max_feat))
df_train['prop_log_historical_price'] = scaler.fit_transform(df_train['prop_log_historical_price'])
df_test['prop_log_historical_price'] = scaler.fit_transform(df_test['prop_log_historical_price'])

df_test = df_test[df_test["price_usd"] < 1000000]